In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Concatenate, Input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras import backend as K

# Focal Loss definition
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())  # To prevent log(0)
        cross_entropy = -y_true * K.log(y_pred)
        modulating_factor = K.pow(1. - y_pred, gamma)
        loss = alpha * modulating_factor * cross_entropy
        return K.sum(loss, axis=1)
    return focal_loss_fixed

# Paths to CSV file and images folder
csv_file = '/content/drive/MyDrive/Soft_Tissue/Final - Facial Profile Types.csv'
image_folder = '/content/drive/MyDrive/Soft_Tissue/cepha400'

# Step 2: Load the CSV file and preprocess
df = pd.read_csv(csv_file)

# Remove "Concave - Convex" class
df = df[df['type'] != 'Concave - Convex']

# Function to get zero-padded image filenames
def get_image_path(image_id, folder):
    image_filename = f"{str(image_id).zfill(3)}.jpg"  # Zero-pad the image ID
    return os.path.join(folder, image_filename)

# Link image paths with tabular data and labels
images = []
labels = []
tabular_features = []

for index, row in df.iterrows():
    image_id = row['data']  # 'data' column
    label = row['type']     # 'type' column
    upper_lip = row['upper_lip']  # 'upper_lip' column
    lower_lip = row['lower_lip']  # 'lower_lip' column

    # Load image with zero-padded filename
    image_path = get_image_path(image_id, image_folder)
    try:
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        images.append(img_array)
        labels.append(label)
        tabular_features.append([upper_lip, lower_lip])
    except FileNotFoundError:
        print(f"Image {image_path} not found.")
        continue

# Convert images and tabular features to NumPy arrays
images = np.array(images)
tabular_features = np.array(tabular_features)

# Encode tabular features using OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
tabular_features_encoded = encoder.fit_transform(tabular_features)

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded)

# Step 3: 5-Fold Cross Validation Setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 1
for train_index, val_index in kf.split(images, labels_encoded):
    print(f"\nTraining fold {fold_no}...")

    # Split the data
    X_train_img, X_val_img = images[train_index], images[val_index]
    X_train_tabular, X_val_tabular = tabular_features_encoded[train_index], tabular_features_encoded[val_index]
    y_train, y_val = labels_one_hot[train_index], labels_one_hot[val_index]

    # Step 4: Data augmentation for training set only
    datagen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=[0.8, 1.2],
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

    # Augment training images
    augmented_images = []
    augmented_labels = []
    augmented_tabular = []

    for idx in range(len(X_train_img)):
        img = X_train_img[idx]
        label = y_train[idx]
        tabular = X_train_tabular[idx]

        img = np.expand_dims(img, axis=0)
        for aug_img in datagen.flow(img, batch_size=1):
            augmented_images.append(aug_img[0])
            augmented_labels.append(label)
            augmented_tabular.append(tabular)
            if len(augmented_images) % 5 == 0:  # Generate 5 augmentations per image
                break

    # Combine original and augmented training data
    X_train_img = np.vstack([X_train_img, np.array(augmented_images)])
    y_train = np.vstack([y_train, np.array(augmented_labels)])
    X_train_tabular = np.vstack([X_train_tabular, np.array(augmented_tabular)])

    print(f"New training set size for fold {fold_no}: {len(X_train_img)} images.")

    # Step 5: Build and compile a multimodal model
    # Image model
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in vgg16.layers:
        layer.trainable = False

    x_img = Flatten()(vgg16.output)

    # Tabular model
    input_tabular = Input(shape=(X_train_tabular.shape[1],), name="tabular_input")
    x_tabular = Dense(128, activation='relu')(input_tabular)

    # Combine image and tabular features
    combined = Concatenate()([x_img, x_tabular])
    x = Dense(256, activation='relu')(combined)
    x = Dropout(0.5)(x)
    output = Dense(y_train.shape[1], activation='softmax')(x)

    model = Model(inputs=[vgg16.input, input_tabular], outputs=output)

    # ASMGrad in Adam optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, amsgrad=True)

    # Compile the model with Focal Loss and ASMGrad optimizer
    model.compile(
        optimizer=optimizer,
        loss=focal_loss(gamma=2., alpha=0.25),
        metrics=['accuracy']
    )

    # Step 6: Train the model for the current fold
    history = model.fit(
        [X_train_img, X_train_tabular], y_train,
        validation_data=([X_val_img, X_val_tabular], y_val),
        epochs=50,
        batch_size=32,
        verbose=1
    )

    # Step 7: Evaluate the model and calculate F1 Score for the current fold
    val_loss, val_accuracy = model.evaluate([X_val_img, X_val_tabular], y_val, verbose=0)
    print(f"Validation Loss for fold {fold_no}: {val_loss:.4f}")
    print(f"Validation Accuracy for fold {fold_no}: {val_accuracy:.4f}")

    # Predict on the validation set
    y_pred = model.predict([X_val_img, X_val_tabular])
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_val, axis=1)

    # Classification Report for the fold
    print("\nClassification Report for fold {fold_no}:")
    print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

    # Calculate F1 Scores for the fold
    macro_f1 = f1_score(y_true_classes, y_pred_classes, average='macro')
    weighted_f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

    print(f"\nMacro-Average F1 Score for fold {fold_no}: {macro_f1:.4f}")
    print(f"Weighted-Average F1 Score for fold {fold_no}: {weighted_f1:.4f}")

    fold_no += 1




Training fold 1...
New training set size for fold 1: 1914 images.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 46s 508ms/step - accuracy: 0.4337 - loss: 0.5452 - val_accuracy: 0.6125 - val_loss: 0.1280
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.6162 - loss: 0.1369 - val_accuracy: 0.6250 - val_loss: 0.1254
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 125ms/step - accuracy: 0.6583 - loss: 0.1138 - val_accuracy: 0.7500 - val_loss: 0.0819
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 127ms/step - accuracy: 0.7881 - loss: 0.0763 - val_accuracy: 0.9125 - val_loss: 0.0523
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 133ms/step - accuracy: 0.8657 - loss: 0.0460 - val_accuracy: 0.8875 - val_loss: 0.0396
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 127ms/step - accuracy: 0.8929 - loss: 0.0332 - val_accuracy: 0.9500 - val_loss: 0.0243
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 129ms/step - accuracy: 0.9156 - loss: 0.0260 - val_accuracy: 0.9750 

2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step

Classification Report for fold {fold_no}:
                  precision    recall  f1-score   support

         Concave       1.00      1.00      1.00        48
          Convex       1.00      0.93      0.97        15
Convex - Concave       1.00      1.00      1.00         4
           Plane       0.93      1.00      0.96        13

        accuracy                           0.99        80
       macro avg       0.98      0.98      0.98        80
    weighted avg       0.99      0.99      0.99        80


Macro-Average F1 Score for fold 3: 0.9821
Weighted-Average F1 Score for fold 3: 0.9875

Training fold 4...
New training set size for fold 4: 1914 images.
Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 181ms/step - accuracy: 0.4731 - loss: 0.4379 - val_accuracy: 0.6000 - val_loss: 0.1369
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 131ms/step - accuracy: 0.5731 - loss: 0.1475 - val_accuracy: 0.6000 - val_loss: 0.1209
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1